#1 부정감정 지속 분석

#1.1 기본 설정

In [1]:
!pip install fsspec==2024.9.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
!pip install datasets==3.1.0 fsspec[http]==2024.9.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.5 MB/s eta 0:00:00


In [3]:
!pip install transformers torch datasets scikit-learn tqdm

#1.2 데이터셋 불러오기

데이터 로드 및 전처리

In [4]:
from datasets import load_dataset
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

# 데이터 로드 (NSMC 데이터셋)
dataset = load_dataset("e9t/nsmc")  # Naver Sentiment Movie Corpus

# 데이터프레임 변환
data = dataset['train'].to_pandas()
data = data.rename(columns={"document": "text", "label": "label"})

# 데이터 중복 제거
data = data.drop_duplicates(subset=['text'])

# 사용자 ID 생성 (1,000명의 사용자 임의 생성)
data['user_id'] = np.random.randint(1, 1001, size=len(data))

# 데이터 분할
train_data, val_data = train_test_split(data, test_size=0.2, stratify=data['label'], random_state=42)

# 데이터 크기 확인
print(f"Train Data Size: {len(train_data)}, Validation Data Size: {len(val_data)}")
print("Train Class Distribution:", train_data['label'].value_counts())
print("Validation Class Distribution:", val_data['label'].value_counts())


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/3.74k [00:00<?, ?B/s]

nsmc.py:   0%|          | 0.00/3.18k [00:00<?, ?B/s]

The repository for e9t/nsmc contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/e9t/nsmc.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/150000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Train Data Size: 116946, Validation Data Size: 29237
Train Class Distribution: label
0    58673
1    58273
Name: count, dtype: int64
Validation Class Distribution: label
0    14669
1    14568
Name: count, dtype: int64


#1.2 모델준비

In [5]:
from transformers import BertTokenizer, BertForSequenceClassification

# KoBERT Tokenizer 및 모델 로드
tokenizer = BertTokenizer.from_pretrained("monologg/kobert")
model = BertForSequenceClassification.from_pretrained("monologg/kobert", num_labels=2)  # 긍정/부정 분류


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/263 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/77.8k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'KoBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.


config.json:   0%|          | 0.00/426 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/369M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


#1.4 데이터로더 및 Pytorch dataset


In [6]:
from torch.utils.data import Dataset, DataLoader
import torch
from transformers import PreTrainedTokenizer

# PyTorch Dataset 클래스 정의
class EmotionDataset(Dataset):
    def __init__(self, data, tokenizer: PreTrainedTokenizer, max_len=128):
        self.texts = data['text'].tolist()
        self.labels = data['label'].tolist()
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(
            text, truncation=True, padding="max_length", max_length=self.max_len, return_tensors="pt"
        )
        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'label': torch.tensor(label, dtype=torch.long)
        }

# DataLoader 생성
train_dataset = EmotionDataset(train_data, tokenizer)
val_dataset = EmotionDataset(val_data, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)


#1.5 Training

In [7]:
from transformers import TrainingArguments, Trainer

# TrainingArguments 설정
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",  # 매 에포크마다 평가
    save_strategy="epoch",        # 매 에포크마다 모델 저장
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    learning_rate=5e-5,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=2  # 저장할 체크포인트 수 제한
)

# 평가지표 함수
def compute_metrics(eval_pred):
    from sklearn.metrics import accuracy_score
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = accuracy_score(labels, predictions)
    return {"accuracy": accuracy}

# Trainer 생성
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# 학습 시작
trainer.train()

# 평가
eval_results = trainer.evaluate()
print(f"Validation Results: {eval_results}")

# 모델 훈련 후 저장
torch.save(model, "trained_baseline_model.pth")
print("Model saved successfully!")

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-7-375295b3cf80>:27: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Accuracy
1,0.689200,0.699105,0.501727
2,0.694000,0.693172,0.498273
3,0.692600,0.693210,0.498273


Validation Results: {'eval_loss': 0.6932100653648376, 'eval_accuracy': 0.49827273660088245, 'eval_runtime': 97.8971, 'eval_samples_per_second': 298.65, 'eval_steps_per_second': 9.336, 'epoch': 3.0}
Model saved successfully!


#2 시계열 분석 + KoBERT

부정 감정 지속성 분석



#2.1 기본 설정

In [8]:
!pip uninstall torch torchvision torchaudio
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121


Found existing installation: torch 2.5.1+cu121
Uninstalling torch-2.5.1+cu121:
  Would remove:
    /usr/local/bin/convert-caffe2-to-onnx
    /usr/local/bin/convert-onnx-to-caffe2
    /usr/local/bin/torchfrtrace
    /usr/local/bin/torchrun
    /usr/local/lib/python3.10/dist-packages/functorch/*
    /usr/local/lib/python3.10/dist-packages/torch-2.5.1+cu121.dist-info/*
    /usr/local/lib/python3.10/dist-packages/torch/*
    /usr/local/lib/python3.10/dist-packages/torchgen/*
Proceed (Y/n)? y
y
y
Y
  Successfully uninstalled torch-2.5.1+cu121
Found existing installation: torchvision 0.20.1+cu121
Uninstalling torchvision-0.20.1+cu121:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/torchvision-0.20.1+cu121.dist-info/*
    /usr/local/lib/python3.10/dist-packages/torchvision.libs/libcudart.7ec1eba6.so.12
    /usr/local/lib/python3.10/dist-packages/torchvision.libs/libjpeg.ceea7512.so.62
    /usr/local/lib/python3.10/dist-packages/torchvision.libs/libnvjpeg.f00ca762.so.12
    /usr/l

#2.2 데이터 전처리 및 모델 학습

In [9]:
import torch
import torch.nn as nn
from datasets import load_dataset
from transformers import BertTokenizer, BertModel, Trainer, TrainingArguments
import numpy as np
from datetime import datetime, timedelta

# 1. 데이터셋 로드 및 전처리
dataset = load_dataset("nsmc")
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

def generate_time_series(num_samples):
    """시계열 데이터 생성"""
    base_time = datetime(2022, 1, 1)
    time_series_data = []
    for i in range(num_samples):
        random_offset = timedelta(minutes=np.random.randint(0, 1440))
        current_time = base_time + random_offset
        time_series_data.append([current_time.hour, current_time.minute])  # (hour, minute)
    return time_series_data

def preprocess_function(examples):
    """코멘트 데이터 전처리 및 시계열 데이터 추가"""
    tokenized_inputs = tokenizer(examples['document'], truncation=True, padding='max_length', max_length=128)
    time_series_data = generate_time_series(len(examples['document']))
    tokenized_inputs['time_series'] = torch.tensor(time_series_data, dtype=torch.float32).unsqueeze(1)  # (batch_size, seq_len=1, input_dim=2)
    tokenized_inputs['labels'] = torch.tensor(examples['label'], dtype=torch.long)
    return tokenized_inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True)
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'time_series', 'labels'])

# 2. 모델 정의
class BertWithTimeSeries(nn.Module):
    def __init__(self):
        super(BertWithTimeSeries, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-multilingual-cased')
        self.classifier = nn.Linear(self.bert.config.hidden_size + 2, 2)  # hidden_size + time series input (2)
        self.loss_fn = nn.CrossEntropyLoss()  # Loss function for classification

    def forward(self, input_ids, attention_mask, time_series, labels=None):
        # BERT 모델 처리
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        # BERT의 마지막 히든 상태 출력 가져오기
        pooled_output = outputs.pooler_output
        # 시간 정보와 결합
        combined_input = torch.cat((pooled_output, time_series.squeeze(1)), dim=1)
        logits = self.classifier(combined_input)

        # 만약 labels가 주어지면 loss를 계산합니다.
        if labels is not None:
            loss = self.loss_fn(logits, labels)
            return loss, logits  # Loss와 logits를 반환
        return logits  # 평가 시에는 logits만 반환

model = BertWithTimeSeries()

# 3. Trainer 설정
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],  # 이미 정의된 데이터셋 사용
    eval_dataset=tokenized_dataset['test'],  # 평가 데이터셋 설정
    data_collator=None,  # 필요에 따라 Data Collator 설정
    compute_metrics=None,  # 필요시 metrics 추가
)

# 4. 학습 시작
trainer.train()

# 평가
eval_results = trainer.evaluate()
print(f"Validation Results: {eval_results}")

# 모델 훈련 후 저장
torch.save(model, "trained_timestamp_model.pth")
print("Model saved successfully!")

README.md:   0%|          | 0.00/3.74k [00:00<?, ?B/s]

nsmc.py:   0%|          | 0.00/3.18k [00:00<?, ?B/s]

The repository for nsmc contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/nsmc.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] 6fa6929f000a6709b626013cea86841854921943
The repository for nsmc contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/nsmc.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/150000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.541200,0.550847
2,0.672200,0.656478
3,0.514600,0.509026


Validation Results: {'eval_loss': 0.509026288986206, 'eval_runtime': 171.8131, 'eval_samples_per_second': 291.014, 'eval_steps_per_second': 4.551, 'epoch': 3.0}
Model saved successfully!


#2.3 학습률 조정 및 early stopping 적용

In [10]:
# 학습률 조정 및 early stopping 적용

import torch.nn as nn
from transformers import BertModel, Trainer, TrainingArguments, EarlyStoppingCallback

# 1. 모델 정의
class BertWithTimeSeries(nn.Module):
    def __init__(self):
        super(BertWithTimeSeries, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-multilingual-cased')
        self.classifier = nn.Linear(self.bert.config.hidden_size + 2, 2)  # hidden_size + time series input (2)
        self.loss_fn = nn.CrossEntropyLoss()  # Loss function for classification

    def forward(self, input_ids, attention_mask, time_series, labels=None):
        # BERT 모델 처리
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        # BERT의 마지막 히든 상태 출력 가져오기
        pooled_output = outputs.pooler_output
        # 시간 정보와 결합
        combined_input = torch.cat((pooled_output, time_series.squeeze(1)), dim=1)
        logits = self.classifier(combined_input)

        # 만약 labels가 주어지면 loss를 계산
        if labels is not None:
            loss = self.loss_fn(logits, labels)
            return {"loss": loss, "logits": logits}
        return {"logits": logits}

# 2. TrainingArguments 설정
training_args = TrainingArguments(
    output_dir="./results",           # 체크포인트 저장 디렉토리
    evaluation_strategy="epoch",      # 매 에포크마다 평가
    save_strategy="epoch",            # 매 에포크마다 저장
    learning_rate=3e-5,               # 학습률 조정
    per_device_train_batch_size=16,   # 학습 배치 크기
    per_device_eval_batch_size=64,    # 평가 배치 크기
    num_train_epochs=10,              # 최대 에포크
    weight_decay=0.01,                # Weight Decay
    logging_dir="./logs",             # 로그 저장 디렉토리
    logging_steps=10,                 # 로깅 간격
    save_total_limit=2                # 저장할 체크포인트 수 제한
)

# 3. Trainer 생성
trainer = Trainer(
    model=model,                      # 사용자 정의 모델
    args=training_args,               # TrainingArguments
    train_dataset=tokenized_dataset['train'],  # 학습 데이터셋
    eval_dataset=tokenized_dataset['test'],    # 평가 데이터셋
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]  # Early Stopping
)

# 4. 학습 시작
trainer.train()

# 모델 훈련 후 저장
torch.save(model, "trained_timestamp_earlystop_model.pth")
print("Model saved successfully!")


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


AssertionError: EarlyStoppingCallback requires load_best_model_at_end = True

#3 Baseline과 증강모델 비교